In [ ]:
import hpmcm
import tables_io
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATADIR = "shear_tract"
shear_st = "0p01"

tract = 10463
SOURCE_TABLEFILES = sorted(glob.glob(os.path.join(DATADIR, f"shear_*_{shear_st}_cleaned_{tract}_ns.pq")))
SOURCE_TABLEFILES.append(os.path.join(DATADIR, f"object_{tract}.pq"))
SOURCE_TABLEFILES.reverse()
VISIT_IDS = np.arange(len(SOURCE_TABLEFILES))

REF_DIR = (37.91729, 6.94389)  # RA, DEC in deg
REGION_SIZE = (1.5, 1.5)
PIXEL_SIZE = 0.5
PIXEL_R2CUT = 4.
PIXEL_MATCH_SCALE = 1

In [ ]:
matcher = hpmcm.WcsMatch.create(REF_DIR, REGION_SIZE, pixSize=PIXEL_SIZE, pixelR2Cut=PIXEL_R2CUT)

In [ ]:
matcher.reduceData(SOURCE_TABLEFILES, VISIT_IDS)

In [ ]:
matcher._nCell

In [ ]:
matcher.analysisLoop()

In [ ]:
cell = matcher.cellDict[(3, 3)]
od = cell.analyze(None, 4)
_ = plt.imshow(od['countsMap'], origin='lower', cmap='gray_r')

In [ ]:
cluster = list(cell._clusterDict.values())[0]
_ = hpmcm.viz_utils.showCluster(od['image'], cluster, cell)

In [ ]:
objLists = matcher.classifyObjects()
hpmcm.Match.printObjectTypes(objLists)

In [ ]:
_ = hpmcm.viz_utils.showShearObj(matcher, objLists['missing'][0])

In [ ]:
_ = plt.scatter(matcher.fullData[2].pgauss_T, matcher.fullData[2].pgauss_psf_T)

In [ ]:
odict = matcher.matchObjectsAgainstRef(snrCut=5.)

In [ ]:
matcher.printObjectMatchTypes(odict)

In [ ]:
_ = plt.hist(matcher.fullData[0].SNR, bins=np.logspace(0, 4, 81), alpha=0.4, label="obj")
_ = plt.hist(matcher.fullData[1].SNR, bins=np.logspace(0, 4, 81), alpha=0.4, label="wmom")
_ = plt.hist(matcher.fullData[2].SNR, bins=np.logspace(0, 4, 81), alpha=0.4, label="pgauss")
_ = plt.hist(matcher.fullData[3].SNR, bins=np.logspace(0, 4, 81), alpha=0.4, label="gauss")
_ = plt.xscale('log')
_ = plt.legend()